# Task 1.4: Compatibility Matrix - SOLUTIONS

This notebook contains solutions to the exercises in the Compatibility Matrix notebook.

---

## Try It Yourself #1 Solution

**Task:** Look up a tool you use frequently. Is it compatible?

In [ ]:
# First, recreate the compatibility infrastructure from the main notebook
from dataclasses import dataclass
from typing import Optional, List
from enum import Enum

class CompatLevel(Enum):
    FULL = "Full Support"
    NGC = "NGC Required"
    PARTIAL = "Partial"
    NO = "Not Compatible"
    UNTESTED = "Untested"

@dataclass
class ToolCompatibility:
    name: str
    category: str
    level: CompatLevel
    notes: str
    workaround: Optional[str] = None
    tested_version: Optional[str] = None
    url: Optional[str] = None

# Sample lookup function
def lookup_tool(name: str, matrix: List[ToolCompatibility]):
    for tool in matrix:
        if name.lower() in tool.name.lower():
            print(f"\n{'='*50}")
            print(f"  {tool.name}")
            print(f"{'='*50}")
            print(f"Category:    {tool.category}")
            print(f"Status:      {tool.level.value}")
            print(f"Notes:       {tool.notes}")
            if tool.workaround:
                print(f"Workaround:  {tool.workaround}")
            return
    print(f"Tool '{name}' not found.")

In [ ]:
# Solution: Example tool lookups

# Create a sample matrix
sample_matrix = [
    ToolCompatibility(
        name="Transformers",
        category="Model Library",
        level=CompatLevel.NGC,
        notes="Works inside NGC PyTorch container. pip install inside container.",
        workaround="docker run --gpus all -it nvcr.io/nvidia/pytorch:25.11-py3 pip install transformers"
    ),
    ToolCompatibility(
        name="LangChain",
        category="Model Library",
        level=CompatLevel.FULL,
        notes="Pure Python - works everywhere. Use with Ollama for local LLM access."
    ),
    ToolCompatibility(
        name="Gradio",
        category="UI Framework",
        level=CompatLevel.FULL,
        notes="Pure Python web framework. Works without modifications."
    ),
]

# Look up common tools
print("Looking up popular tools...\n")
lookup_tool("transformers", sample_matrix)
lookup_tool("langchain", sample_matrix)
lookup_tool("gradio", sample_matrix)

---

## Challenge Solution

**Task:** Add 5 more tools to the compatibility matrix.

In [ ]:
# Solution: 5 additional tools for the compatibility matrix

additional_tools = [
    ToolCompatibility(
        name="Streamlit",
        category="UI Framework",
        level=CompatLevel.FULL,
        notes="Pure Python web framework. Works without modifications.",
        tested_version="1.32+",
        url="https://streamlit.io"
    ),
    ToolCompatibility(
        name="FastAPI",
        category="Web Framework",
        level=CompatLevel.FULL,
        notes="Pure Python ASGI framework. Works perfectly for serving ML models.",
        tested_version="0.110+",
        url="https://fastapi.tiangolo.com"
    ),
    ToolCompatibility(
        name="Sentence Transformers",
        category="Model Library",
        level=CompatLevel.NGC,
        notes="Requires PyTorch with CUDA. Use NGC container.",
        workaround="pip install sentence-transformers inside NGC container",
        tested_version="2.6+"
    ),
    ToolCompatibility(
        name="llama-cpp-python",
        category="LLM Inference",
        level=CompatLevel.PARTIAL,
        notes="Python bindings for llama.cpp. Must compile with CUDA flags for GPU support.",
        workaround="CMAKE_ARGS='-DGGML_CUDA=on' pip install llama-cpp-python",
        url="https://github.com/abetlen/llama-cpp-python"
    ),
    ToolCompatibility(
        name="OpenCV",
        category="Computer Vision",
        level=CompatLevel.NGC,
        notes="GPU-accelerated OpenCV available in NGC containers.",
        workaround="Use nvcr.io/nvidia/pytorch container which includes OpenCV",
        tested_version="4.9+"
    ),
]

# Display the new tools
print("Additional Tools for DGX Spark Compatibility Matrix")
print("=" * 60)
print(f"{'Tool':<25} {'Category':<20} {'Status':<15}")
print("-" * 60)

for tool in additional_tools:
    print(f"{tool.name:<25} {tool.category:<20} {tool.level.value:<15}")

In [ ]:
# Solution: How to research compatibility for new tools

def research_tool_compatibility(tool_name: str):
    """
    Guide for researching DGX Spark compatibility of a new tool.
    """
    print(f"\nResearching compatibility for: {tool_name}")
    print("=" * 50)
    print()
    print("Step 1: Check if it's pure Python")
    print(f"   pip show {tool_name.lower().replace(' ', '-')}")
    print("   Look for 'Requires:' - if only Python packages, likely works")
    print()
    print("Step 2: Check PyPI for ARM64 wheels")
    print(f"   https://pypi.org/project/{tool_name.lower().replace(' ', '-')}/#files")
    print("   Look for 'aarch64' or 'arm64' in wheel filenames")
    print()
    print("Step 3: Check NGC Catalog")
    print("   https://catalog.ngc.nvidia.com/")
    print(f"   Search for '{tool_name}'")
    print()
    print("Step 4: Check GitHub Issues")
    print(f"   Search: '{tool_name} aarch64' or '{tool_name} ARM64'")
    print()
    print("Step 5: Check Docker Hub for multi-arch images")
    print(f"   docker manifest inspect <image> | grep arm64")

# Example research
research_tool_compatibility("Your Tool Name")

In [ ]:
# Solution: Export the extended matrix
import json
from datetime import datetime

def export_extended_matrix(tools: list, filename: str = "extended_compatibility_matrix.json"):
    """Export the extended compatibility matrix to JSON."""
    data = {
        "generated": datetime.now().isoformat(),
        "platform": "NVIDIA DGX Spark",
        "note": "Extended matrix with user-added tools",
        "tools": [
            {
                "name": t.name,
                "category": t.category,
                "status": t.level.name,
                "notes": t.notes,
                "workaround": t.workaround,
                "tested_version": t.tested_version,
                "url": t.url
            }
            for t in tools
        ]
    }
    
    with open(filename, 'w') as f:
        json.dump(data, f, indent=2)
    
    print(f"Saved to {filename}")
    return filename

# Export the additional tools
export_extended_matrix(additional_tools)

---

## Quick Reference: Compatibility Patterns

| Pattern | Likely Status | Example |
|---------|--------------|----------|
| Pure Python (no C extensions) | Full Support | LangChain, Gradio |
| Has aarch64 wheels on PyPI | Full Support | NumPy, Pandas |
| Requires PyTorch/TensorFlow | NGC Required | Transformers, Diffusers |
| Has custom CUDA kernels | NGC or Partial | vLLM, DeepSpeed |
| x86-only binaries | Not Compatible | Some legacy tools |

---

## Key Takeaways

1. **Pure Python packages** almost always work
2. **PyTorch-dependent packages** need NGC containers
3. **Check NGC first** before attempting compilation
4. **ARM64 wheels** are increasingly available on PyPI
5. **Document your findings** to help the community

---

## Cleanup

In [ ]:
# Cleanup
import gc
gc.collect()
print("Cleanup complete!")